In [4]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

# 모델 초기화
model = ChatOpenAI(
    temperature=0,  # 응답 다양성 설정, 0~2, 기본값은 0.7
    model='gpt-3.5-turbo'
)

# 프롬프트 템플릿 정의
restaurant_template = """
You are a tourist guide. You know many good restaurants around the tourist destination.
You can recommend good foods and restaurants.
Here is a question:
{input}
"""

transport_template = """
You are a tourist guide. You have a lot of knowledge in public transportation.
You can provide information about public transportation to help tourists get to the tourist destination.
Here is a question:
{input}
"""

destination_template = """
You are a tourist guide. You know many good tourist places.
You can recommend good tourist places to the tourists.
Here is a question:
{input}
"""

# 프롬프트 정보 리스트
prompt_infos = [
    {
        "name": "restaurants",
        "description": "Good for recommending restaurants around the tourist destinations",
        "prompt_template": restaurant_template
    },
    {
        "name": "transport",
        "description": "Good for guiding the transport to get the place",
        "prompt_template": transport_template
    },
    {
        "name": "destination",
        "description": "Good for recommending place to tour",
        "prompt_template": destination_template
    }
]

# 각 체인을 생성하고 딕셔너리에 추가
destination_chains = {}
for prompt_info in prompt_infos:
    name = prompt_info["name"]
    prompt = PromptTemplate.from_template(prompt_info["prompt_template"])
    chain = LLMChain(llm=model, prompt=prompt, verbose=True)
    destination_chains[name] = chain

# 기본 체인 설정
default_prompt = PromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=model, prompt=default_prompt)

# 라우터 프롬프트 템플릿 설정
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

# Router Chain 초기화
router_chain = LLMRouterChain.from_llm(model, router_prompt)

# MultiPromptChain 설정
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, 
                         verbose=True
                        )


/home/restful3/anaconda3/envs/trading/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/home/restful3/anaconda3/envs/trading/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new MultiPromptChain chain...
restaurants: {'input': 'What is the best restaurant in Seoul?'}

> Entering new LLMChain chain...
Prompt after formatting:

You are a tourist guide. You know many good restaurants around the tourist destination.
You can recommend good foods and restaurants.
Here is a question:
What is the best restaurant in Seoul?


> Finished chain.

> Finished chain.
There are many amazing restaurants in Seoul, but one that is highly recommended is Jungsik. It is a Michelin-starred restaurant known for its innovative and modern take on Korean cuisine. The chef, Yim Jung-sik, combines traditional Korean flavors with modern techniques to create a truly unique dining experience. The tasting menu is a must-try for food enthusiasts looking to explore the best of Korean cuisine. Make sure to make a reservation in advance as it is a popular spot among locals and tourists alike.


- 레스토랑 추천 관련 질문

In [5]:
response = chain.run("What is the best restaurant in Seoul?")
print(response)

# response = chain.run("Can you recommend a good sushi place in Tokyo?")
# print(response)

# response = chain.run("Where can I find the best pasta in Rome?")
# print(response)



> Entering new MultiPromptChain chain...
restaurants: {'input': 'What is the best restaurant in Seoul?'}

> Entering new LLMChain chain...
Prompt after formatting:

You are a tourist guide. You know many good restaurants around the tourist destination.
You can recommend good foods and restaurants.
Here is a question:
What is the best restaurant in Seoul?


> Finished chain.

> Finished chain.
There are many amazing restaurants in Seoul, but one that is highly recommended is Jungsik. It is a Michelin-starred restaurant known for its innovative and modern take on Korean cuisine. The chef, Yim Jungsik, combines traditional Korean flavors with modern techniques to create a unique dining experience. The restaurant offers a tasting menu that showcases the best of Korean ingredients and flavors. If you are looking for a truly special dining experience in Seoul, Jungsik is definitely worth a visit.


- 교통 안내 관련 질문

In [6]:
response = chain.run("How can I get to the Eiffel Tower from Charles de Gaulle airport?")
print(response)

# response = chain.run("What is the best way to travel from JFK airport to Times Square?")
# print(response)

# response = chain.run("Is there a direct train from London to Edinburgh?")
# print(response)



> Entering new MultiPromptChain chain...
transport: {'input': 'How can I get from Charles de Gaulle airport to the Eiffel Tower?'}

> Entering new LLMChain chain...
Prompt after formatting:

You are a tourist guide. You have a lot of knowledge in public transportation.
You can provide information about public transportation to help tourists get to the tourist destination.
Here is a question:
How can I get from Charles de Gaulle airport to the Eiffel Tower?


> Finished chain.

> Finished chain.
To get from Charles de Gaulle airport to the Eiffel Tower, you have a few options for public transportation. 

1. RER B Train: The RER B train line connects Charles de Gaulle airport to central Paris. You can take the RER B train from the airport to the Saint-Michel-Notre-Dame station and then transfer to the RER C train line to get to the Champ de Mars-Tour Eiffel station, which is the closest station to the Eiffel Tower.

2. Roissybus: Another option is to take the Roissybus, which is a dire

- 관광지 추천 관련 질문

In [7]:
response = chain.run("What are the must-see places in Paris?")
print(response)

# response = chain.run("Can you suggest some good tourist spots in New York City?")
# print(response)

# response = chain.run("What are the best attractions to visit in Tokyo?")
# print(response)



> Entering new MultiPromptChain chain...
destination: {'input': 'What are the must-see places in Paris?'}

> Entering new LLMChain chain...
Prompt after formatting:

You are a tourist guide. You know many good tourist places.
You can recommend good tourist places to the tourists.
Here is a question:
What are the must-see places in Paris?


> Finished chain.

> Finished chain.
There are so many must-see places in Paris, but some of the top recommendations include:

1. The Eiffel Tower - Iconic symbol of Paris and offers stunning views of the city from the top.

2. The Louvre Museum - Home to thousands of works of art, including the famous Mona Lisa.

3. Notre Dame Cathedral - A beautiful example of French Gothic architecture, although currently under restoration after the 2019 fire.

4. Montmartre and Sacré-Cœur Basilica - Charming neighborhood with a bohemian vibe and a stunning white-domed basilica at the top of the hill.

5. Champs-Élysées and Arc de Triomphe - Famous avenue lined 

- 기타 질문 (기본 체인으로 처리)
    - 기본 체인은 특정 카테고리에 속하지 않는 질문들을 처리합니다.

In [9]:
response = chain.run("What is the capital of France?")
print(response)

# response = chain.run("Tell me about the history of the Great Wall of China.")
# print(response)

# response = chain.run("Who won the Nobel Prize in Literature in 2020?")
# print(response)



> Entering new MultiPromptChain chain...
None: {'input': 'What is the capital of France?'}
> Finished chain.
The capital of France is Paris.
